## Import Package

In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import *
from gurobipy import GRB


## Define Data

### 1. Cost matrices for products at different nodes.

### 2. Availability and storage capacity of products at different nodes.

### 3. Distance Matrices

### 4. Demand matrices for products at LDCs and customers locations

#### Demand at LDCs

In [32]:
demandLDC = [
    {"demand": "l1", "dc": "j1", "total": 50},
    {"demand": "l1", "dc": "j2", "total": 100},
    {"demand": "l1", "dc": "j3", "total": 40},
    {"demand": "l1", "dc": "j4", "total": 50},
    {"demand": "l1", "dc": "j5", "total": 120},
    {"demand": "l2", "dc": "j1", "total": 140},
    {"demand": "l2", "dc": "j2", "total": 20},
    {"demand": "l2", "dc": "j3", "total": 300},
    {"demand": "l2", "dc": "j4", "total": 150},
    {"demand": "l2", "dc": "j5", "total": 60},
    {"demand": "l3", "dc": "j1", "total": 200},
    {"demand": "l3", "dc": "j2", "total": 130},
    {"demand": "l3", "dc": "j3", "total": 44},
    {"demand": "l3", "dc": "j4", "total": 100},
    {"demand": "l3", "dc": "j5", "total": 40},
]
demandLDC = pd.DataFrame(demandLDC)

# Buat tabel demand pivot
demand_pivot = demandLDC.pivot_table(index="dc", columns="demand", values="total")

# Tampilkan tabel demandLDC
print(demand_pivot)

demand   l1   l2   l3
dc                   
j1       50  140  200
j2      100   20  130
j3       40  300   44
j4       50  150  100
j5      120   60   40


#### Demand at Customer Location

In [33]:
demandCL = [
    {"demand": "l1", "dc": "k1", "total": 50},
    {"demand": "l1", "dc": "k2", "total": 10},
    {"demand": "l1", "dc": "k3", "total": 40},
    {"demand": "l1", "dc": "k4", "total": 50},
    {"demand": "l1", "dc": "k5", "total": 12},
    {"demand": "l2", "dc": "k1", "total": 14},
    {"demand": "l2", "dc": "k2", "total": 20},
    {"demand": "l2", "dc": "k3", "total": 30},
    {"demand": "l2", "dc": "k4", "total": 15},
    {"demand": "l2", "dc": "k5", "total": 60},
    {"demand": "l3", "dc": "k1", "total": 20},
    {"demand": "l3", "dc": "k2", "total": 13},
    {"demand": "l3", "dc": "k3", "total": 44},
    {"demand": "l3", "dc": "k4", "total": 10},
    {"demand": "l3", "dc": "k5", "total": 40},
]
demandCL = pd.DataFrame(demandCL)

# Buat tabel demand pivot
demand_pivot = demandCL.pivot_table(index="dc", columns="demand", values="total")

# Tampilkan tabel demandLDC
print(demand_pivot)

demand  l1  l2  l3
dc                
k1      50  14  20
k2      10  20  13
k3      40  30  44
k4      50  15  10
k5      12  60  40


### 5. Number of vehicles required for the delivery and returned pickup of orders

## Create Optimization Model

In [3]:
minlp_model = Model(name= "MINLP")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-03


## Decision Variables


$X_{ijl}^{FL1}$ = x1\
$X_{jkl}^{FL2}$ = x2\
$X_{jkl}^{Ret,RL}$ = x3\
$X_{mil}^{Resell,RL}$ = x4\
$X_{mfl}^{Refurb,RL}$ = x5\
$X_{mel}^{Recy,RL}$ = x6

---
**Quantity**:\
$Q_{ijl}^{FL1}$ = q1\
$Q_{jkl}^{FL2}$ = q2\
$Q_{kml}^{Ret,RL}$ = q3\
$Q_{mil}^{Resell,RL}$ = q4\
$Q_{mfl}^{Refurb,RL}$ = q5\
$Q_{mel}^{Recy,RL}$ = q6

---
**Product Selling Price at Supplier Point**:\
$SP_{il}^{FL}$ = sp

---
**Cost**:\
$C_{il}^{pc}$ = c1\
$C_{ml}^{pc,resell}$ = c2\
$C_{ml}^{pc,refurb}$ = c3\
$C_{ml}^{pc,recy}$ = c4\
$C^{tpc,FL}$ = c5\
$C^{lmdc,FL}$ = c6\
$C^{ct,lv}$ = c7\
$C^{ct,sv}$ = c8\
$C^{hc}$ = c9\
$C^{tpc,RL}$ = c10\
$C^{lmpc,RL}$ = c11

In [4]:
x1 = minlp_model.addVar(vtype=GRB.BINARY, name="XFL1ijl")
x2 = minlp_model.addVar(vtype=GRB.BINARY, name="XFL2jkl")
x3 = minlp_model.addVar(vtype=GRB.BINARY, name="XRet,RL")
x4 = minlp_model.addVar(vtype=GRB.BINARY, name="XResell,RL")
x5 = minlp_model.addVar(vtype=GRB.BINARY, name="XRefurb,RL")
x6 = minlp_model.addVar(vtype=GRB.BINARY, name="XRecy,RL")


In [5]:
q1 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="QFL1ijl")
q2 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="QFL2jkl")
q3 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="QRet,RL")
q4 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="QResell,RL")
q5 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="QRefurb,RL")
q6 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="QRecy,RL")

In [6]:
sp = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="SP FLil")

In [7]:
c1 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="CPC,il")
c2 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="CPC,resell")
c3 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="CPC,refurb")
c4 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="CPC,recy")
c5 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="Ctpc,fl")
c6 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="Clmdc,fl")
c7 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="Cct,lv")
c8 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="Cct,sv")
c9 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="Chc")
c10 = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="Clmpc,rl")

## Objective Function

MAX[Profit Forward + Profit Backward]\
MAX[revenue-cost Forward + revenue-cost Backward]

**MAX**[selling price(s)-cost(forward)+selling price(resellable)+selling price(refurbished)+selling price(recycle)-cost(reverse)]


In [8]:
z = minlp_model.addVar(vtype=GRB.CONTINUOUS, name="Z")

In [9]:
for xyz in profit:
    minlp_model.addConstr(
        quicksum(quicksum(x[i,j,k]*dist_matrix[i,j] for j in total)for i in total) <= z,
    )

NameError: name 'profit' is not defined

## Constraints

## Solve Model

## Output the Results

In [38]:
import random
import gurobipy as grb
import math

n = 10
Distance = 25

def distance(points, i, j):
  dx = points[i][0] - points[j][0]
  dy = points[i][1] - points[j][1]
  return math.sqrt(dx*dx + dy*dy)

random.seed(1)
points = []
for i in range(n):
  points.append((random.randint(0,100),random.randint(0,100)))
opt_model = grb.Model(name="MILP Model")

# <= Variables

x_vars = {}
for i in range(n):
   for j in range(n):
     x_vars[i,j] = opt_model.addVar(vtype=grb.GRB.BINARY,
                          name='e'+str(i)+'_'+str(j))
u={}
for i in range(1,n):
    u[i]=opt_model.addVar(vtype=grb.GRB.INTEGER,
                          name='e'+str(i))

# <= Constraint (Mandatory Edges and excluding vertexes) Eq(1)

opt_model.addConstr((grb.quicksum(x_vars[1,j] for j in range(1,n)))  == 1)
opt_model.addConstr((grb.quicksum(x_vars[i,(n-1)] for i in range(n-1)))  == 1)
opt_model.addConstr((grb.quicksum(x_vars[i,i] for i in range(n-1)))  == 0)
# <= Constraint (Distance) Eq(3)
for i in range(n-1):
  opt_model.addConstr((grb.quicksum(x_vars[i,j]*distance(points, i, j) for j in range(1,n) for i in range(n-1))) <= Distance)


# <= Constraint (Equality & Single edge in and out) Eq(2)

for k in range(1, n-1):
  opt_model.addConstr(grb.quicksum(x_vars[i,k] for i in range(n-1))
                      == grb.quicksum(x_vars[k,j] for j in range(1, n)) <=1)
for k in range(1, n-1):
  opt_model.addConstr(grb.quicksum(x_vars[i,k] for i in range(n-1))
                       <=1)
  for k in range(1, n - 1):
      opt_model.addConstr(grb.quicksum(x_vars[k, j] for j in range(1, n))
                          <= 1)

# <= Constraint (Subtour elimination) Eq(4) Eq(5)

for i in range(1,n):
  opt_model.addConstr(2 <= u[i] <= n)
for i in range(1,n):
    for j in range(1,n):
        opt_model.addConstr((u[i] - u[j] +1 <= (n-1)*(1-x_vars[j,i])))

# <= objective (maximize) Eq(1)

objective = grb.quicksum(x_vars[i,j]
                         for i in range(1, n-1)
                         for j in range(1, n))

opt_model.ModelSense = grb.GRB.MAXIMIZE
opt_model.setObjective(objective)
opt_model.update()
opt_model.optimize()
solution = opt_model.getAttr('x', x_vars )
select = grb.tuplelist((i,j) for i,j in x_vars.keys() if x_vars[i,j].X > 0.5)

for i in range(len(select)):
    print(points[select[i][0]],points[select[i][1]])
    print(distance(points,select[i][0],select[i][1]))

GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)